In [655]:
import numpy as np
from random import seed
from random import random
from math import exp

In [656]:
def initialize_matrix_network(n_inputs, n_neurons , n_outputs):
	network = list()
	hidden_layer1 = {}
	hidden_layer1['weights'] =  np.random.random((n_inputs,n_neurons))
	network.append(hidden_layer1)
	hidden_layer2  = {}
	hidden_layer2['weights'] =  np.random.random((n_neurons,n_neurons))
	network.append(hidden_layer2)
	output_layer  = {}
	output_layer['weights'] =   np.random.random((n_neurons,n_outputs))
	network.append(output_layer)
	return network

In [657]:
inputFilenameWithPath = 'train_data.txt'
inputData = np.loadtxt(inputFilenameWithPath, delimiter=",")
#print('Number of training examples: '+ str(len(inputData)))
#n_inputs = len(inputData[0]) - 1
n_outputs = len(set([row[-1] for row in inputData]))
#print('Number of features: '+ str(n_inputs) + " Number of classes:" + str(n_outputs))

network = initialize_matrix_network(n_inputs, 3, n_outputs)

In [658]:
def sigmoid_derivative(z):
    return z*(1-z)

In [659]:
def softmax(z):
    sum = np.sum(np.exp(z), axis=1, keepdims=True)
    return np.divide(np.exp(z),sum)

In [660]:
def forward_propagate(network, inputData):
    layerInput = inputData
    #print(layerInput.shape)
    sigmoid = np.vectorize(lambda x: 1.0 / (1.0 + np.exp(-x)))
    #First hidden layer
    layerIterator = 0;
    for layer in network:
        z = np.dot(layerInput, layer['weights']) #Just for the input layer
        #z = np.dot(layerInput, layer['weights']) #For the other layers
        layer['z'] = z
        #print("z",z)
        if(layerIterator == len(network) - 1):
            layer['output'] = softmax(z)
        else:
            layer['output'] = sigmoid(z)
        print("a",layer['output'])
        layerInput = layer['output'] #[1 x n_neurons]
        layerIterator += 1
    return network[-1]['output']
    
forward_propagate(network, inputData[12:13,:2])
print(network[2]['output'])

('a', array([[ 0.66231308,  0.6948515 ,  0.44575596]]))
('a', array([[ 0.70518283,  0.71919247,  0.63793602]]))
('a', array([[ 0.45322474,  0.54677526]]))
[[ 0.45322474  0.54677526]]


In [661]:
def softmax_derivative(y1, y2):
    return y1
    #return y1(1 - y1) - y1*y2

In [662]:
def error_output(y_predicted, y_actual):
    return 2*(y_predicted - y_actual)

In [663]:
def backward_propagate(network,prediction, trueValue):
    layerInput = prediction * np.ones(n_outputs)
    truth = trueValue  * np.ones(n_outputs)
    print("Prediction",layerInput)
    print("Truth",truth)
    #error = np.vectorize(lambda x: error_output(outputData ,x))
    dy_dz_operation = np.vectorize(lambda x: softmax_derivative(x, 1-x));
    sigmoid_derivative_operation = np.vectorize(lambda x: sigmoid_derivative(x))
    for i in reversed(range(len(network))):
        dL_dy = network[i]['error'] = (trueValue - layerInput)
        if(i == len(network) - 1):
            print('The Error - dL_dy:', dL_dy)
            print("Loss:", np.sum(dL_dy**2))
            dy_dz = dy_dz_operation(network[i]['output']);
            delta = network[i]['delta'] = dL_dy * dy_dz
            #print('delta', delta)
            network[i]['weight_update'] = layerInput = dL_dw = np.outer(delta, network[i-1]['output'])
            
        else:
            #print('delta.shape:', delta.shape)
            #print('network[i+1][weights].shape:', network[i+1]['weights'].T.shape)
            wT_dot_delta = np.dot(delta, network[i+1]['weights'].T)
            #print('wT_dot_delta.shape:', wT_dot_delta.shape)
            delta = network[i]['delta'] = wT_dot_delta * sigmoid_derivative_operation(network[i]['z'])
            network[i]['weight_update'] = layerInput = dL_dw =  np.outer(delta , network[i-1]['output']) 
        #print(delta)
        #print('layerInput.shape:', layerInput.shape)
        #layerInput = dL_dw = np.outer(delta, network[i-1]['output'])
        #print('dL_dw.shape',dL_dw.shape)
        #layerInput = error * 
        
#start_index = 500  
#prediction = forward_propagate(network, inputData[start_index:start_index + 1,:2])   
#print(prediction)
#backward_propagate(network, prediction,inputData[start_index:start_index + 1,2])
#update_weights(network, eta)

In [664]:
def error_hidden(H, Wo, Eo):
    # H * (1-H) * (E . Wo^T)
    return np.multiply(np.multiply(H,(1 - H)), Eo.dot(Wo.T))

In [675]:
def update_weights(network, eta):
    for i in reversed(range(len(network))):
        update = network[i]['weight_update']
        #print('update.shape',update.shape)
        #print('network[i][weights].shape',network[i]['weights'].shape)
        network[i]['weights'] -= update*eta
        #print(network[i]['weights'])
#eta = 0.1
#update_weights(network, eta)

In [666]:
def predict(inputRow,network):
    return forward_propagate(network, inputRow)

#predict(inputData[:1,:2], network)

In [671]:
n_epochs = 50
eta = 0.05

def runSGD(inputData,network):
    n_inputs = len(inputData[0]) - 1
    network = initialize_matrix_network(n_inputs, 3, n_outputs)
    n_iterations = 50
    for epoch in range(n_epochs): #For each epoch
        for i in range(n_iterations):
            random_index = np.random.randint(n_iterations)
            inputRow = inputData[random_index:random_index+1,:]
            #print(inputRow)
            prediction = forward_propagate(network, inputRow[:,:2])
            backward_propagate(network,prediction,inputRow[:,2])
            print("Prediction:",prediction," Truth:",inputRow[:,2])
            update_weights(network, eta)
            #print(network)

In [676]:
runSGD(inputData,network)

('a', array([[ 0.802256  ,  0.86178891,  0.53165097]]))
('a', array([[ 0.77082761,  0.80720876,  0.80864198]]))
('a', array([[ 0.5343925,  0.4656075]]))
('Prediction', array([[ 0.5343925,  0.4656075]]))
('Truth', array([ 1.,  1.]))
('The Error - dL_dy:', array([[ 0.4656075,  0.5343925]]))
('Loss:', 0.50236568812978832)
('Prediction:', array([[ 0.5343925,  0.4656075]]), ' Truth:', array([ 1.]))


ValueError: operands could not be broadcast together with shapes (3,2) (2,3) (3,2) 